In [1]:
!pip install geopy
import pandas as pd 
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim
import requests
print('Libraries imported.')


Libraries imported.


In [57]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
df=pd.read_csv('sorted_geoloc.csv')

In [3]:
df.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M5H,DowntownToronto,"Adelaide,King,Richmond",43.650571,-79.384568
1,1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577
3,3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
4,4,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484


In [4]:
latitude = df['Latitude']
longitude = df['Longitude']


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [6]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
Scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()
df.drop('Unnamed: 0',1,inplace = True)

In [11]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [12]:
# create map of Toronto using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'H2LKECAZUVAQTPDKAU5HUY0R4EFRQOQWLIF2AO4KIAHM0AOZ' # your Foursquare ID
CLIENT_SECRET = 'CASUDFBPFXTYKGTYIP3I3R510XERSWUUYJGVY41ZMROQKL5U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H2LKECAZUVAQTPDKAU5HUY0R4EFRQOQWLIF2AO4KIAHM0AOZ
CLIENT_SECRET:CASUDFBPFXTYKGTYIP3I3R510XERSWUUYJGVY41ZMROQKL5U


### Let's explore the first neighborhood in our dataframe

In [14]:
Scarborough_data.loc[0, 'Neighbourhood']

'Agincourt'

In [15]:
n_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
n_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

n_name = Scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(n_name,n_latitude, 
                                                               n_longitude))
                               

Latitude and longitude values of Agincourt are 43.7942003, -79.26202940000002.


## Now, let's get the top 100 venues that are in Agincourt within a radius of 500 meters

In [16]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_latitude, 
    n_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=H2LKECAZUVAQTPDKAU5HUY0R4EFRQOQWLIF2AO4KIAHM0AOZ&client_secret=CASUDFBPFXTYKGTYIP3I3R510XERSWUUYJGVY41ZMROQKL5U&v=20180605&ll=43.7942003,-79.26202940000002&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e21d3d2660a9f0025f084e6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Agincourt',
  'headerFullLocation': 'Agincourt, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7987003045, 'lng': -79.25580688126533},
   'sw': {'lat': 43.7897002955, 'lng': -79.2682519187347}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e3451fcd4c063434821c41e',
       'name': "Panagio's Breakfast & Lunch",
       'location': {'address': '1928 McCowan Rd.',
        'lat': 43.79237001092506,
        'lng': -79.26020325805978,
        'labeledLatLngs': [{'label':

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Panagio's Breakfast & Lunch,Breakfast Spot,43.792370,-79.260203
1,El Pulgarcito,Latin American Restaurant,43.792648,-79.259208
2,Twilight,Lounge,43.791999,-79.258584


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Scarborough

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighbourhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Agincourt
Agincourt North,L'Amoreaux East,Milliken,Steeles East
Birch Cliff,Cliffside West
Cedarbrae
Clairlea,Golden Mile,Oakridge
Clarks Corners,Sullivan,Tam O'Shanter
Cliffcrest,Cliffside,Scarborough Village West
Dorset Park,Scarborough Town Centre,Wexford Heights
East Birchmount Park,Ionview,Kennedy Park
Guildwood,Morningside,West Hill
Highland Creek,Rouge Hill,Port Union
L'Amoreaux West
Maryvale,Wexford
Rouge,Malvern
Scarborough Village
Upper Rouge
Woburn


In [23]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(92, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
1,Agincourt,43.794200,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant
2,Agincourt,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge
3,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,Queen Pastisserie,43.815492,-79.289715,Bakery
4,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,Port Royal Park,43.815477,-79.289773,Park


In [24]:
Scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,3,3,3,3,3,3
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",14,14,14,14,14,14
"Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2
"Dorset Park,Scarborough Town Centre,Wexford Heights",6,6,6,6,6,6
"East Birchmount Park,Ionview,Kennedy Park",5,5,5,5,5,5


In [25]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 52 uniques categories.


## 3. Analyze Each Neighborhood

In [26]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighbourhood'] = Scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
Scarborough_onehot.shape

(92, 53)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
Scarborough_grouped


,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000,0.000000,0.333333,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.25,0.0,0.000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.125000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,...,0.0,0.000,0.000000,0.000000,0.00,0.1,0.000,0.000000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.0,0.000,0.000000,0.000000,0.071429,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.071429,0.00,0.0,0.000,0.000000,0.071429,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.166667
8,"East Birchmount Park,Ionview,Kennedy Park",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
9,"Guildwood,Morningside,West Hill",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.125000,0.0,0.0,...,0.0,0.125,0.000000,0.000000,0.00,0.0,0.125,0.000000,0.000000,0.000000


### Let's confirm the new size

In [29]:
Scarborough_grouped.shape

(16, 53)

In [30]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.33
1                     Lounge  0.33
2             Breakfast Spot  0.33
3                   Pharmacy  0.00
4         Light Rail Station  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0               Bakery  0.33
1                 Park  0.33
2           Playground  0.33
3  American Restaurant  0.00
4             Pharmacy  0.00


----Birch Cliff,Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4               Pharmacy  0.00


----Cedarbrae----
                venue  freq
0     Thai Restaurant  0.12
1              Bakery  0.12
2                Bank  0.12
3  Athletics & Sports  0.12
4    Hakka Restaurant  0.12


----Clairlea,Golden Mile,Oakridge----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2   Intersecti

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Scarborough_grouped['Neighbourhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Bakery,Playground,Park,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Park,College Stadium,General Entertainment


### 4. Cluster Neighborhoods

In [33]:
# set number of clusters
kclusters = 5

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')



In [39]:
Scarborough_merged.dropna(inplace = True)

In [47]:
Scarborough_merged.head() # check the last columns!
Scarborough_merged['Cluster Labels'] = [round(x) for x in Scarborough_merged['Cluster Labels']]

In [48]:
Scarborough_merged.drop('Unnamed: 0',1,inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [51]:
latitude = 43.773077
longitude = -79.257774
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighbourhood'],Scarborough_merged['Cluster Labels'] ):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

### Cluster 1

In [52]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


### Cluster 2

In [53]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Latin American Restaurant,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,Scarborough,1,Bakery,Playground,Park,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
3,Scarborough,1,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store
4,Scarborough,1,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Park,College Stadium,General Entertainment
5,Scarborough,1,Pizza Place,Intersection,Fried Chicken Joint,Noodle House,Convenience Store,Pharmacy,Fast Food Restaurant,Shopping Mall,Bank,Italian Restaurant
7,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Light Rail Station,Pet Store,Bank,Bar,Grocery Store,General Entertainment,Gas Station
8,Scarborough,1,Discount Store,Coffee Shop,Department Store,Convenience Store,Thai Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,Scarborough,1,Intersection,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,Gas Station
11,Scarborough,1,Grocery Store,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Sandwich Place,Supermarket,Breakfast Spot,Pizza Place,Pharmacy,Gas Station


### Cluster 3

In [54]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,American Restaurant,Motel,Indian Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


### Cluster 4

In [55]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,3,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Cluster 5

In [56]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 4, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Scarborough,4,Coffee Shop,Indian Restaurant,Korean Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
